In [1]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import string
import glob
import os

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
##Read the training data in the folder
positive_com = glob.glob(r'C:\Users\kangh\Desktop\TakeHome - 1 - NLP\TakeHome - 1 - NLP\datasets\Train\pos\*.txt')

negative_com = glob.glob(r'C:\Users\kangh\Desktop\TakeHome - 1 - NLP\TakeHome - 1 - NLP\datasets\Train\neg\*.txt')

pos_list = []

for i in positive_com:
    file = open(i,'r',encoding="ISO-8859-1")
    str_1 = file.readline()
    pos_list.append(str_1)
    
neg_list = []
for i in negative_com:
    file = open(i,'r',encoding="ISO-8859-1")
    str_2 = file.readline()
    neg_list.append(str_2)

In [4]:
##Turn the read training data into a dataframe
##Set 1 as positive and -1 as negative
df_positive = pd.DataFrame()

df_positive['comment'] = pos_list
df_positive['classification'] = 1
df_negative = pd.DataFrame()
df_negative['comment'] = neg_list
df_negative['classification'] = -1

entire_df = pd.concat([df_positive,df_negative], axis = 0)
entire_df.index=(range(25000))

In [5]:
##add the ratio of the punctuation in the sentence as another new feature
stopwords = nltk.corpus.stopwords.words('english')
ps = ps = nltk.PorterStemmer()
from sklearn.feature_extraction.text import CountVectorizer

##add the length of the sentence as a new feature
entire_df['LENGTH'] = entire_df['comment'].apply(lambda x:len(x) - x.count(' '))


##define a function that count the punctuation in the sentence and calculate the ratio over the sentence
def count_punc(text):
    count = sum([1 for char in text if char in string.punctuation])
    return 100*round(count/(len(text)-text.count(' ')),3)

##add the ratio of the punctuation in the sentence as another new feature
entire_df['punctuation%'] = entire_df['comment'].apply(lambda x:count_punc(x))

##define a function that remove '<br />' in the sentence
##remove the punctuation
##tokenize
##PorterStemmer
def clean_text(text):
    text = re.sub("<br />",'',text)
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+',text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

##Use Tfidf Vectorizer with the function clean_text as the analyzer
Tfidf_vect = TfidfVectorizer(analyzer = clean_text)
x_count = Tfidf_vect.fit_transform(entire_df['comment'])

In [6]:
##Since the dataset is too large for my computer to run, I turn the processed data into a csr matrix,
##therefore my computer would be able to run it. 
from scipy.sparse import hstack
from scipy.sparse import csr_matrix
sparse_dataset = csr_matrix(x_count)
s = entire_df[['LENGTH','punctuation%']]
sparse_dataset_length_punc = csr_matrix(s)
combined_sparse_dataset = hstack((sparse_dataset_length_punc,sparse_dataset))

## RANDOM FOREST CLASSIFIER

In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold, cross_val_score
rf = RandomForestClassifier(n_jobs = 2)
k_fold = KFold(n_splits = 5)
cross_val_score(rf,combined_sparse_dataset,entire_df['classification'],cv = k_fold, scoring = 'accuracy',n_jobs = 2)

array([0.3694, 0.3784, 0.7396, 0.5964, 0.6136])

In [8]:
##import test data from the test folder
positive_com_test = glob.glob(r'C:\Users\kangh\Desktop\TakeHome - 1 - NLP\TakeHome - 1 - NLP\datasets\Test\pos\*.txt')

negative_com_test = glob.glob(r'C:\Users\kangh\Desktop\TakeHome - 1 - NLP\TakeHome - 1 - NLP\datasets\Test\neg\*.txt')

pos_list_test = []

for i in positive_com_test:
    file = open(i,'r',encoding="ISO-8859-1")
    str_1 = file.readline()
    pos_list_test.append(str_1)
    
neg_list_test = []

for i in negative_com_test:
    file = open(i,'r',encoding="ISO-8859-1")
    str_2 = file.readline()
    neg_list_test.append(str_2)

In [9]:
##turn the read test data into a dataframe
df_positive_test = pd.DataFrame()

df_positive_test['comment'] = pos_list_test
df_positive_test['classification'] = 1

df_negative_test = pd.DataFrame()
df_negative_test['comment'] = neg_list_test
df_negative_test['classification'] = -1

entire_df_test = pd.concat([df_positive_test,df_negative_test], axis = 0)

entire_df_test.index = range(25000) #reindex the index


In [10]:
##process the test data
entire_df_test['LENGTH'] = entire_df_test['comment'].apply(lambda x:len(x) - x.count(' '))
entire_df_test['punctuation%'] = entire_df_test['comment'].apply(lambda x:count_punc(x))

x_count_test = Tfidf_vect.transform(entire_df_test['comment'])


In [11]:
##csr-matrix the data in order to allow my computer to run it
sparse_dataset_test = csr_matrix(x_count_test)
s_test = entire_df[['LENGTH','punctuation%']]
sparse_dataset_length_punc_test = csr_matrix(s_test)
combined_sparse_dataset_test = hstack((sparse_dataset_length_punc_test,sparse_dataset))

In [12]:
##define the training and testing dataset
x_train = combined_sparse_dataset
y_train = entire_df['classification']
x_test = combined_sparse_dataset_test
y_test = entire_df_test['classification']

In [13]:
##Conduct GridSearch
from sklearn.model_selection import GridSearchCV
param = {
    'n_estimators' : [10,100,1000],
}

gs = GridSearchCV(rf,param,cv=5,n_jobs=-1)
gs_fit = gs.fit(combined_sparse_dataset,entire_df['classification']) 
pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score',ascending=False)[0:5]

C:\Users\kangh\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\kangh\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\kangh\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\kangh\Anaconda3\lib\site-packages\sklearn\util

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
2,1644.808613,155.038392,4.127562,0.765638,1000,{'n_estimators': 1000},0.8456,0.8514,0.8566,0.8432,...,0.85004,0.004953,1,1.0000,1.0000,1.00000,1.0000,1.0000,1.00000,0.000000
1,270.453651,16.140311,1.315086,0.179383,100,{'n_estimators': 100},0.8288,0.8290,0.8484,0.8252,...,0.83392,0.008420,2,1.0000,1.0000,1.00000,1.0000,1.0000,1.00000,0.000000
0,23.327612,1.735813,0.529783,0.078314,10,{'n_estimators': 10},0.7350,0.7506,0.7408,0.7258,...,0.73788,0.008062,3,0.9939,0.9937,0.99395,0.9945,0.9944,0.99409,0.000307


In [14]:
#build a random forest model
rf = RandomForestClassifier(n_estimators = 1000, max_depth = 20)
rf_model = rf.fit(x_train,y_train)
y_predict = rf_model.predict(x_test)

In [15]:
from sklearn.metrics import precision_recall_fscore_support as score

In [16]:
precision,recall,fscore,support = score(y_test,y_predict,pos_label = 1, average = None)

In [17]:
print('precision :{}, recall:{},accuracy:{}'.format(precision,recall,(y_predict == y_test).sum()/len(y_predict)))

precision :[0.96982575 0.91764261], recall:[0.9128 0.9716],accuracy:0.9422


## LOGISTIC REGRESSION

In [18]:
from sklearn.linear_model import LogisticRegression

In [19]:
##Conduct GridSearch
param_lr = {'C' : [1,2,3,4], 'penalty':['l1','l2']}

gs = GridSearchCV(LogisticRegression(),param_lr,cv=5,n_jobs=-1)
gs_fit = gs.fit(combined_sparse_dataset,entire_df['classification']) 
pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score',ascending=False)[0:5]

C:\Users\kangh\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\kangh\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\kangh\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\kangh\Anaconda3\lib\site-packages\sklearn\util

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_penalty,params,split0_test_score,split1_test_score,split2_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,1.604710,0.178846,0.006383,0.001038,1,l1,"{'C': 1, 'penalty': 'l1'}",0.8636,0.8576,0.8570,...,0.85976,0.003143,1,0.89300,0.89710,0.89455,0.89215,0.89315,0.89399,0.001735
1,6.122630,1.897212,0.007381,0.000798,1,l2,"{'C': 1, 'penalty': 'l2'}",0.8674,0.8490,0.8508,...,0.85812,0.007198,2,0.93305,0.93745,0.93615,0.93500,0.93450,0.93523,0.001491
2,4.282155,0.350500,0.006983,0.000631,2,l1,"{'C': 2, 'penalty': 'l1'}",0.8640,0.8514,0.8484,...,0.85692,0.005969,3,0.92205,0.92605,0.92520,0.92355,0.92240,0.92385,0.001555
3,9.322084,0.669217,0.006384,0.000490,2,l2,"{'C': 2, 'penalty': 'l2'}",0.8664,0.8444,0.8476,...,0.85460,0.007891,4,0.95265,0.95525,0.95505,0.95460,0.95290,0.95409,0.001097
5,9.539504,0.801571,0.005785,0.000399,3,l2,"{'C': 3, 'penalty': 'l2'}",0.8654,0.8396,0.8450,...,0.85180,0.008922,5,0.96295,0.96940,0.96640,0.96555,0.96450,0.96576,0.002153


In [20]:
##Build a Logistic Regression model
lr = LogisticRegression(random_state=0,C=1, penalty = 'l1')
lr_model = lr.fit(x_train,y_train)
y_predict_lr = lr_model.predict(x_test)

In [21]:
precision,recall,fscore,support = score(y_test,y_predict_lr,pos_label = 1, average = None)

In [22]:
print('precision :{}, recall:{},accuracy:{}'.format(precision,recall,(y_predict_lr == y_test).sum()/len(y_predict)))

precision :[0.90791745 0.88649996], recall:[0.88344 0.9104 ],accuracy:0.89692


## KNN

In [23]:
from sklearn.neighbors import KNeighborsClassifier

In [24]:
##Conduct GridSearch for KNN
param_knn = {
    'n_neighbors' : [5,10]
}

gsknn = GridSearchCV(KNeighborsClassifier(),param_knn,cv=5,n_jobs=2)
gs_knn = gsknn.fit(combined_sparse_dataset,entire_df['classification'])
pd.DataFrame(gs_knn.cv_results_).sort_values('mean_test_score',ascending=False)[0:5]

C:\Users\kangh\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\kangh\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\kangh\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\kangh\Anaconda3\lib\site-packages\sklearn\util

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_neighbors,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,0.222184,0.106058,10.583667,3.845678,5,{'n_neighbors': 5},0.5186,0.5220,0.5240,0.5168,...,0.51932,0.003254,1,0.69765,0.69410,0.6973,0.70060,0.7020,0.69833,0.002758
1,0.195569,0.068424,8.259686,1.056118,10,{'n_neighbors': 10},0.5130,0.5176,0.5188,0.5228,...,0.51708,0.003678,2,0.63620,0.63395,0.6366,0.63415,0.6353,0.63524,0.001061


In [25]:
#Build a KNN model
KNN_classifier = KNeighborsClassifier(n_neighbors=5)  
KNN_classifier.fit(x_train, y_train) 
y_predict_knn = KNN_classifier.predict(x_test)

In [26]:
precision,recall,fscore,support = score(y_test,y_predict_knn,pos_label = 1, average = None)
print('precision :{}, recall:{},accuracy:{}'.format(precision,recall,(y_predict_knn == y_test).sum()/len(y_predict)))

precision :[0.69898893 0.70094059], recall:[0.7024  0.69752],accuracy:0.69996
